# Assorted helpers

In [ ]:
#| default_exp helpers

In [ ]:
#| export

from web3 import Web3, constants
from typing import List
from decimal import Decimal
from datetime import datetime, timedelta
from dataclasses import dataclass

In [ ]:
#| export

def normalize_address(address: str) -> str: return Web3.to_checksum_address(address.lower())

ADDRESS_ZERO = constants.ADDRESS_ZERO
MAX_UINT256 = Web3.to_int(hexstr='0x' + 'f' * 64)

# def is_address(value: str) -> bool:
#     return Web3.is_address(value)


# def cache_in_seconds(seconds: int):
#     return alru_cache(ttl=seconds)

def chunk(list_to_chunk: List, n: int):
    for i in range(0, len(list_to_chunk), n):
        yield list_to_chunk[i : i + n]


def amount_to_k_string(amount: float) -> str:
    """Turns 2000 to "2K" """
    return f"{round(amount/1000, 2)}K"


def format_currency(value: float, symbol: str = "$", prefix: bool = True) -> str:
    v = "{:0,.2f}".format(value)
    return f"{symbol}{v}" if prefix else f"{v} {symbol}"


def format_percentage(value: float) -> str:
    return "{:0,.2f} %".format(value)


def amount_to_m_string(amount: float) -> str:
    """Turns 2000000 to "2M" """
    return f"{round(amount/1000000, 2)}M"

In [ ]:
#| export

def float_to_uint256(amount: float, decimals: int = 18) -> int:
    """Convert float to uint256 with decimal scaling"""
    # Convert float to Decimal for precision
    amount_decimal = Decimal(str(amount))
    # Scale by decimals
    scaled_amount = amount_decimal * Decimal(10 ** decimals)
    # Convert to integer
    return int(scaled_amount)

In [ ]:
float_to_uint256(1.0, 18)

1000000000000000000

In [ ]:
#| export

def get_future_timestamp(deadline_minutes: float) -> int:
    """Convert minutes from now to future unix timestamp"""
    future_time = datetime.now() + timedelta(minutes=deadline_minutes)
    return int(future_time.timestamp())

In [ ]:
get_future_timestamp(5)

1741886626

In [ ]:
#| export

def apply_slippage(amount: int, slippage: float) -> int:
    if slippage < 0 or slippage > 1: raise ValueError("Slippage must be between 0 and 1")
    return int(amount * (1 - slippage))

## Graph department

This is where all the nerds hang out

In [ ]:
#| export
# Claude 3.7 sonnet made this

@dataclass
class Pair: token0: str; token1: str

def find_all_paths(pairs, a, b, max_depth=3):
    """
    Find paths from token 'a' to token 'b' in a graph defined by pairs, with limits to prevent 
    excessive computation in dense, cyclical graphs.
    
    Parameters:
    pairs (list): A list of dictionaries with 'token0' and 'token1' keys
    a (str): Starting token
    b (str): Destination token
    max_depth (int): Maximum path length to explore
    
    Returns:
    list: A list of paths, where each path is a list of tokens
    """
    # Build adjacency list representation of the graph
    graph = {}
    for pair in pairs:
        token0, token1 = pair.token0, pair.token1
        
        # Add connections in both directions for an undirected graph
        if token0 not in graph:
            graph[token0] = []
        if token1 not in graph:
            graph[token1] = []
            
        graph[token0].append(token1)
        graph[token1].append(token0)
    
    # Check if start or end nodes exist in the graph
    if a not in graph or b not in graph:
        return []
    
    # List to store all found paths
    all_paths = []
    
    # Helper function for DFS traversal with depth limit
    def dfs(current, destination, visited, path, depth=0):    
        # If we've exceeded our maximum depth, stop this branch
        if depth >= max_depth:
            return
            
        # Add current node to path
        path.append(current)
        
        # If we've reached the destination, add the path to our results
        if current == destination:
            all_paths.append(path.copy())
        else:
            # Mark current node as visited to avoid cycles
            visited.add(current)
            
            # Explore neighbors
            for neighbor in graph[current]:
                if neighbor not in visited:
                    # Create copies of visited and path for this branch of exploration
                    dfs(neighbor, destination, visited.copy(), path.copy(), depth + 1)
    
    # Start DFS from the starting token
    dfs(a, b, set(), [])
    
    # clear duplicates (merge paths using "-", run this through a set, then split by "-")
    return [p.split("-") for p in set(["-".join(p) for p in all_paths])]
    

In [ ]:
 # Example pairs - including a cycle
pairs = [
    Pair("A", "B"),
    Pair("B", "C"),
    Pair("A", "D"),
    Pair("D", "C"),
    Pair("B", "D"),
    Pair("C", "E"),
    Pair("E", "A")  # This creates a cycle A -> E -> C -> B -> A
]

# Find paths from A to C
paths = find_all_paths(pairs, "A", "C")

print(f"All paths from A to C:")
for i, path in enumerate(paths):
    print(f"Path {i+1}: {' -> '.join(path)}")
    
# Let's also test with a different example involving a cycle
cycle_pairs = [
    Pair("X", "Y"),
    Pair("Y", "Z"),
    Pair("Z", "X")
]

# Find paths from X to Z
cycle_paths = find_all_paths(cycle_pairs, "X", "Z")

print(f"\nAll paths from X to Z in a cyclical graph:")
for i, path in enumerate(cycle_paths):
    print(f"Path {i+1}: {' -> '.join(path)}")

All paths from A to C:
Path 1: A -> B -> C
Path 2: A -> E -> C
Path 3: A -> D -> C

All paths from X to Z in a cyclical graph:
Path 1: X -> Z
Path 2: X -> Y -> Z


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()